topics check

In [ ]:
# Cell 1: List all topics (name, type, count) in a ROS1 bag

from pathlib import Path
from collections import Counter
from rosbags.highlevel import AnyReader

bag_path = Path("/home/afaris/Final_Project/labels/Passing Conversational Groups/bag files/A_Spot_Bass_Rec_Fri_Nov_26_128 (copy).bag")

# Open the bag and count messages per (topic, type)
counters = Counter()
topic_types = {}
with AnyReader([bag_path]) as reader:
    for conn, _, _ in reader.messages():
        counters[conn.topic] += 1
        topic_types[conn.topic] = conn.msgtype

# Print a neat summary
print(f"\nBag: {bag_path}\n" + "="*70)
for topic, count in sorted(counters.items()):
    print(f"Topic: {topic:<45} Type: {topic_types[topic]:<35} Messages: {count}")



Bag: /home/afaris/Final_Project/labels/Passing Conversational Groups/bag files/A_Spot_Bass_Rec_Fri_Nov_26_128 (copy).bag
Topic: /image_raw/compressed                         Type: sensor_msgs/msg/CompressedImage     Messages: 15580
Topic: /joint_states                                 Type: sensor_msgs/msg/JointState          Messages: 8496
Topic: /joystick                                     Type: sensor_msgs/msg/Joy                 Messages: 31216
Topic: /localization                                 Type: amrl_msgs/msg/Localization2DMsg     Messages: 8496
Topic: /navigation/cmd_vel                           Type: geometry_msgs/msg/Twist             Messages: 20857
Topic: /odom                                         Type: nav_msgs/msg/Odometry               Messages: 8495
Topic: /scan                                         Type: sensor_msgs/msg/LaserScan           Messages: 5169
Topic: /spot/camera/back/camera_info                 Type: sensor_msgs/msg/CameraInfo          Messages: 

## Main pre processing pipeline

In [ ]:
import cv2
import numpy as np
import pandas as pd
import json
import csv
import shutil
from pathlib import Path
from rosbags.highlevel import AnyReader
from scipy.signal import find_peaks

# CONFIGURATION 

# 1. input directory containing all clean Jackal bag files
INPUT_BAG_DIR = Path("/home/afaris/Final_Project/newly_downloaded_jackal_bags")

# 2. names for the two main output directories
GEMINI_OUTPUT_DIR = Path("./gemini_bundles")
LOCAL_MODEL_OUTPUT_DIR = Path("./local_model_bundles")

# 3. Parameters for event detection and keyframe extraction
EVENT_WINDOW_SECONDS = 3.0
EVENT_THRESHOLD_PERCENTILE = 95.0
NUM_KEYFRAMES = 6
CLEANUP_INTERMEDIATE_FILES = True

# HELPER FUNCTIONS

def find_topics(bag_path: Path):
    with AnyReader([bag_path]) as reader:
        topics_in_bag = {c.topic for c in reader.connections}
    if "/jackal_velocity_controller/odom" in topics_in_bag and "/camera/rgb/image_raw/compressed" in topics_in_bag:
        return {'image_topic': "/camera/rgb/image_raw/compressed", 'odom_topic': "/jackal_velocity_controller/odom"}
    return None

def extract_data_to_csv(bag_path, image_topic, odom_topic, temp_dir):
    image_csv_path = temp_dir / "images.csv"
    odom_csv_path = temp_dir / "odom.csv"
    with open(image_csv_path, 'w', newline='') as img_f, open(odom_csv_path, 'w', newline='') as odom_f:
        img_writer, odom_writer = csv.writer(img_f), csv.writer(odom_f)
        img_writer.writerow(['timestamp'])
        odom_writer.writerow(['timestamp', 'linear_x', 'angular_z'])
        with AnyReader([bag_path]) as reader:
            connections = [c for c in reader.connections if c.topic in [image_topic, odom_topic]]
            for conn, ts, rawdata in reader.messages(connections=connections):
                ts_sec = ts / 1e9
                if conn.topic == image_topic:
                    img_writer.writerow([ts_sec])
                elif conn.topic == odom_topic:
                    msg = reader.deserialize(rawdata, conn.msgtype)
                    odom_writer.writerow([ts_sec, msg.twist.twist.linear.x, msg.twist.twist.angular.z])
    return image_csv_path, odom_csv_path

def synchronize_data(image_csv_path, odom_csv_path):
    image_df, odom_df = pd.read_csv(image_csv_path), pd.read_csv(odom_csv_path)
    if odom_df.empty or image_df.empty: return pd.DataFrame()
    img_ts, odom_ts, odom_vels = image_df['timestamp'].to_numpy(), odom_df['timestamp'].to_numpy(), odom_df[['linear_x', 'angular_z']].to_numpy()
    sync_data = []
    for t_img in img_ts:
        idx_after = np.searchsorted(odom_ts, t_img)
        idx_before = idx_after - 1
        if idx_before < 0 or idx_after >= len(odom_ts): continue
        t_before, v_before = odom_ts[idx_before], odom_vels[idx_before]
        t_after, v_after = odom_ts[idx_after], odom_vels[idx_after]
        ratio = (t_img - t_before) / (t_after - t_before) if (t_after - t_before) != 0 else 0
        v_interp = v_before + ratio * (v_after - v_before)
        sync_data.append({'timestamp': t_img, 'linear_x': v_interp[0], 'angular_z': v_interp[1]})
    return pd.DataFrame(sync_data)

def create_full_video(bag_path, image_topic, temp_dir):
    timestamps = []
    width, height = 0, 0
    with AnyReader([bag_path]) as reader:
        connections = [c for c in reader.connections if c.topic == image_topic]
        for conn, ts, rawdata in reader.messages(connections=connections):
            timestamps.append(ts / 1e9)
            if width == 0:
                msg = reader.deserialize(rawdata, conn.msgtype)
                frame = cv2.imdecode(np.frombuffer(msg.data, np.uint8), cv2.IMREAD_COLOR)
                if frame is not None: height, width, _ = frame.shape
    if not timestamps: return None, 0, 0, 0
    avg_fps = 1.0 / np.mean(np.diff(timestamps)) if len(timestamps) > 1 else 30.0
    video_path = temp_dir / "full_video.mp4"
    writer = cv2.VideoWriter(str(video_path), cv2.VideoWriter_fourcc(*'mp4v'), avg_fps, (width, height))
    with AnyReader([bag_path]) as reader:
        connections = [c for c in reader.connections if c.topic == image_topic]
        for conn, ts, rawdata in reader.messages(connections=connections):
            msg = reader.deserialize(rawdata, conn.msgtype)
            frame = cv2.imdecode(np.frombuffer(msg.data, np.uint8), cv2.IMREAD_COLOR)
            if frame is not None: writer.write(frame)
    writer.release()
    return video_path, avg_fps, width, height

def find_event_windows(sync_df, fps):
    if sync_df.empty: return []
    df = sync_df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.set_index('timestamp', inplace=True)
    rolling_std = df['angular_z'].rolling(f'{EVENT_WINDOW_SECONDS}s').std().dropna()
    if rolling_std.empty: return []
    threshold = np.percentile(rolling_std, EVENT_THRESHOLD_PERCENTILE)
    distance = int(EVENT_WINDOW_SECONDS * fps)
    peaks, _ = find_peaks(rolling_std, height=threshold, distance=distance)
    return [rolling_std.index[p] for p in peaks]

def create_bundles_for_event(full_video_path, sync_df, event_end_ts, fps, width, height, bundle_id):
    event_start_ts = event_end_ts - pd.Timedelta(seconds=EVENT_WINDOW_SECONDS)
    
    # Create Gemini Bundle (6 Keyframes + JSON)
    gemini_bundle_path = GEMINI_OUTPUT_DIR / f"b{bundle_id}"
    gemini_bundle_path.mkdir(exist_ok=True)
    
    target_timestamps = [event_start_ts + pd.Timedelta(seconds=(i / (NUM_KEYFRAMES - 1)) * EVENT_WINDOW_SECONDS) for i in range(NUM_KEYFRAMES)]
    
    cap = cv2.VideoCapture(str(full_video_path))
    video_start_time_sec = sync_df['timestamp'].iloc[0]
    gemini_commands = {}
    
    frame_idx, target_keyframe_idx = 0, 0
    frames_to_save = {} # Store frame index and frame data
    
    while cap.isOpened() and target_keyframe_idx < NUM_KEYFRAMES:
        ret, frame = cap.read()
        if not ret: break
        current_frame_ts = video_start_time_sec + (frame_idx / fps)
        next_target_ts = target_timestamps[target_keyframe_idx].timestamp()
        if current_frame_ts >= next_target_ts:
            frames_to_save[target_keyframe_idx] = frame
            target_keyframe_idx += 1
        frame_idx += 1
        
    for idx, frame in frames_to_save.items():
        frame_name = f"frame_{idx}.jpg"
        cv2.imwrite(str(gemini_bundle_path / frame_name), frame)
        closest_row = sync_df.iloc[(sync_df['timestamp'] - target_timestamps[idx].timestamp()).abs().idxmin()]
        gemini_commands[frame_name] = {'linear_x': closest_row['linear_x'], 'angular_z': closest_row['angular_z']}

    with open(gemini_bundle_path / 'commands.json', 'w') as f: json.dump(gemini_commands, f, indent=2)
    
    # Create Local Model Bundle (Video Clip + CSV) 
    local_bundle_path = LOCAL_MODEL_OUTPUT_DIR / f"b{bundle_id}"
    local_bundle_path.mkdir(exist_ok=True)
    
    # Save the full command data for the 3-second window
    event_commands_df = sync_df[(sync_df['timestamp'] >= event_start_ts.timestamp()) & (sync_df['timestamp'] <= event_end_ts.timestamp())]
    event_commands_df.to_csv(local_bundle_path / 'commands.csv', index=False)
    
    # Create the 3-second video clip
    clip_path = local_bundle_path / 'event_video.mp4'
    clip_writer = cv2.VideoWriter(str(clip_path), cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    start_frame = int((event_start_ts.timestamp() - video_start_time_sec) * fps)
    end_frame = int((event_end_ts.timestamp() - video_start_time_sec) * fps)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, start_frame))
    for i in range(max(0, start_frame), end_frame):
        ret, frame = cap.read()
        if not ret: break
        clip_writer.write(frame)
        
    clip_writer.release()
    cap.release()

# MAIN WORKFLOW
if __name__ == "__main__":
    GEMINI_OUTPUT_DIR.mkdir(exist_ok=True)
    LOCAL_MODEL_OUTPUT_DIR.mkdir(exist_ok=True)
    
    jackal_bag_files = sorted(list(INPUT_BAG_DIR.glob("*Jackal*.bag")))
    print(f"{'='*20} Found {len(jackal_bag_files)} Jackal bag files to process {'='*20}")
    
    global_bundle_counter = 1
    
    for bag_path in jackal_bag_files:
        print(f"\n--- Processing Bag: {bag_path.name} ---")
        temp_dir = INPUT_BAG_DIR / f"temp_{bag_path.stem}"
        temp_dir.mkdir(exist_ok=True)
        
        topics = find_topics(bag_path)
        if not topics: continue
        
        img_csv, odom_csv = extract_data_to_csv(bag_path, topics['image_topic'], topics['odom_topic'], temp_dir)
        sync_df = synchronize_data(img_csv, odom_csv)
        if sync_df.empty:
            print("  -> Skipping bag due to empty synchronized data.")
            if CLEANUP_INTERMEDIATE_FILES: shutil.rmtree(temp_dir)
            continue
            
        full_video_path, fps, width, height = create_full_video(bag_path, topics['image_topic'], temp_dir)
        if not full_video_path:
            print("  -> Skipping bag due to video creation failure.")
            if CLEANUP_INTERMEDIATE_FILES: shutil.rmtree(temp_dir)
            continue
            
        event_timestamps = find_event_windows(sync_df, fps)
        print(f"  -> Found {len(event_timestamps)} significant events in this bag.")
        
        for event_ts in event_timestamps:
            create_bundles_for_event(full_video_path, sync_df, event_ts, fps, width, height, global_bundle_counter)
            print(f"    - Created Bundle b{global_bundle_counter} for both pipelines.")
            global_bundle_counter += 1
            
        if CLEANUP_INTERMEDIATE_FILES:
            shutil.rmtree(temp_dir)
            
    print(f"\n{'='*20} PIPELINE COMPLETE {'='*20}")
    print(f"Total Jackal bags processed: {len(jackal_bag_files)}")
    print(f"Total bundles created: {global_bundle_counter - 1}")
    print(f"Gemini bundles saved in: {GEMINI_OUTPUT_DIR}")
    print(f"Local model bundles saved in: {LOCAL_MODEL_OUTPUT_DIR}")

==================== Found 29 Jackal bag files to process ====================

--- Processing Bag: A_Jackal_AHG_Library_Thu_Nov_4_16.bag ---

--- Processing Bag: A_Jackal_AHG_Library_Thu_Oct_28_1.bag ---
  -> Found 9 significant events in this bag.
    - Created Bundle b1 for both pipelines.
    - Created Bundle b2 for both pipelines.
    - Created Bundle b3 for both pipelines.
    - Created Bundle b4 for both pipelines.
    - Created Bundle b5 for both pipelines.
    - Created Bundle b6 for both pipelines.
    - Created Bundle b7 for both pipelines.
    - Created Bundle b8 for both pipelines.
    - Created Bundle b9 for both pipelines.

--- Processing Bag: A_Jackal_AHG_Library_Thu_Oct_28_2.bag ---
  -> Found 8 significant events in this bag.
    - Created Bundle b10 for both pipelines.
    - Created Bundle b11 for both pipelines.
    - Created Bundle b12 for both pipelines.
    - Created Bundle b13 for both pipelines.
    - Created Bundle b14 for both pipelines.
    - Created Bundle 

In [ ]:
import cv2
import numpy as np
import pandas as pd
import json
import csv
import shutil
from pathlib import Path
from rosbags.highlevel import AnyReader
from scipy.signal import find_peaks

# 1. directory containing all clean Jackal bag files
INPUT_BAG_DIR = Path("/home/afaris/Final_Project/newly_downloaded_jackal_bags")

# 2. name for the single output directory
OUTPUT_DIR = Path("./sliding_window_bundles")

# 3. Parameters
EVENT_WINDOW_SECONDS = 3.0
EVENT_THRESHOLD_PERCENTILE = 95.0
NUM_KEYFRAMES_PER_EVENT = 6
NUM_PREV_FRAMES_INPUT = 6
CLEANUP_INTERMEDIATE_FILES = True


def find_topics(bag_path: Path):
    with AnyReader([bag_path]) as reader:
        topics_in_bag = {c.topic for c in reader.connections}
    if "/jackal_velocity_controller/odom" in topics_in_bag and "/camera/rgb/image_raw/compressed" in topics_in_bag:
        return {'image_topic': "/camera/rgb/image_raw/compressed", 'odom_topic': "/jackal_velocity_controller/odom"}
    return None

def extract_and_save_data(bag_path, image_topic, odom_topic, temp_dir):
    # Streams odom data to a CSV and saves every compressed image frame to a temporary directory, named by its timestamp.

    odom_csv_path = temp_dir / "odom.csv"
    temp_frames_path = temp_dir / "frames"
    temp_frames_path.mkdir(exist_ok=True)
    
    image_timestamps = []

    with open(odom_csv_path, 'w', newline='') as odom_f:
        odom_writer = csv.writer(odom_f)
        odom_writer.writerow(['timestamp', 'linear_x', 'angular_z'])
        with AnyReader([bag_path]) as reader:
            connections = [c for c in reader.connections if c.topic in [image_topic, odom_topic]]
            for conn, ts, rawdata in reader.messages(connections=connections):
                ts_sec = ts / 1e9
                if conn.topic == image_topic:
                    image_timestamps.append(ts_sec)
                    # Write the raw compressed image data directly to a file
                    with open(temp_frames_path / f"{ts_sec:.6f}.jpg", 'wb') as img_file:
                        msg = reader.deserialize(rawdata, conn.msgtype)
                        img_file.write(msg.data)
                elif conn.topic == odom_topic:
                    msg = reader.deserialize(rawdata, conn.msgtype)
                    odom_writer.writerow([ts_sec, msg.twist.twist.linear.x, msg.twist.twist.angular.z])

    # Create a simple CSV of image timestamps for the synchronization step
    image_csv_path = temp_dir / "images.csv"
    pd.DataFrame(image_timestamps, columns=['timestamp']).to_csv(image_csv_path, index=False)
    
    return image_csv_path, odom_csv_path, temp_frames_path

def synchronize_data(image_csv_path, odom_csv_path):
    image_df, odom_df = pd.read_csv(image_csv_path), pd.read_csv(odom_csv_path)
    if odom_df.empty or image_df.empty: return pd.DataFrame()
    img_ts, odom_ts, odom_vels = image_df['timestamp'].to_numpy(), odom_df['timestamp'].to_numpy(), odom_df[['linear_x', 'angular_z']].to_numpy()
    sync_data = []
    for t_img in img_ts:
        idx_after = np.searchsorted(odom_ts, t_img)
        idx_before = idx_after - 1
        if idx_before < 0 or idx_after >= len(odom_ts): continue
        t_before, v_before = odom_ts[idx_before], odom_vels[idx_before]
        t_after, v_after = odom_ts[idx_after], odom_vels[idx_after]
        ratio = (t_img - t_before) / (t_after - t_before) if (t_after - t_before) != 0 else 0
        v_interp = v_before + ratio * (v_after - v_before)
        sync_data.append({'timestamp': t_img, 'linear_x': v_interp[0], 'angular_z': v_interp[1]})
    return pd.DataFrame(sync_data)

# The function now accepts the configuration variables
def find_event_windows(sync_df, event_window_seconds, event_threshold_percentile):
    if sync_df.empty: return []
    # We need an approximate FPS to calculate distance for peak finding
    avg_fps = 1.0 / np.mean(np.diff(sync_df['timestamp'])) if len(sync_df) > 1 else 30.0
    
    df = sync_df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.set_index('timestamp', inplace=True)
    rolling_std = df['angular_z'].rolling(f'{event_window_seconds}s').std().dropna()
    if rolling_std.empty: return []
    threshold = np.percentile(rolling_std, event_threshold_percentile)
    distance = int(event_window_seconds * avg_fps)
    peaks, _ = find_peaks(rolling_std, height=threshold, distance=distance)
    return [rolling_std.index[p] for p in peaks]

def create_sliding_window_bundles(sync_df, temp_frames_path, event_end_ts, bundle_id):
    # For a 3-second event, creates 6 sub-bundles by copying the 6 frames pREVIOUS to the corresponding keyframe from the temporary frames folder.
    
    event_start_ts = event_end_ts - pd.Timedelta(seconds=EVENT_WINDOW_SECONDS)
    bundle_path = OUTPUT_DIR / f"b{bundle_id}"
    bundle_path.mkdir(exist_ok=True)
    
    keyframes_ts = [event_start_ts + pd.Timedelta(seconds=(i / (NUM_KEYFRAMES_PER_EVENT - 1)) * EVENT_WINDOW_SECONDS) for i in range(NUM_KEYFRAMES_PER_EVENT)]
    
    # Get a sorted list of all saved frame paths
    saved_frame_paths = sorted(list(temp_frames_path.glob("*.jpg")))
    if not saved_frame_paths: return # Skip if no frames were saved
    saved_frame_timestamps = np.array([float(p.stem) for p in saved_frame_paths])

    for i, key_ts in enumerate(keyframes_ts):
        keyframe_input_path = bundle_path / f"keyframe_{i}_input"
        keyframe_input_path.mkdir(exist_ok=True)
        
        # Find the index of the keyframe in the full list of frames
        keyframe_idx = np.searchsorted(saved_frame_timestamps, key_ts.timestamp())
        
        # Determine the start index for the N PREVIOUS frames
        start_idx = max(0, keyframe_idx - NUM_PREV_FRAMES_INPUT)
        
        # Get the paths for the frames we need to copy
        frames_to_copy = saved_frame_paths[start_idx:keyframe_idx]
        
        for frame_count, frame_path in enumerate(frames_to_copy):
            dest_name = f"prev_frame_{frame_count}.jpg"
            shutil.copy(frame_path, keyframe_input_path / dest_name)

# MAIN WORKFLOW 
if __name__ == "__main__":
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    jackal_bag_files = sorted(list(INPUT_BAG_DIR.glob("*Jackal*.bag")))
    print(f"{'='*20} Found {len(jackal_bag_files)} Jackal bag files to process {'='*20}")
    
    global_bundle_counter = 1
    
    for bag_path in jackal_bag_files:
        print(f"\n--- Processing Bag: {bag_path.name} ---")
        temp_dir = INPUT_BAG_DIR / f"temp_{bag_path.stem}"
        temp_dir.mkdir(exist_ok=True)
        
        topics = find_topics(bag_path)
        if not topics: continue
            
        img_csv, odom_csv, temp_frames = extract_and_save_data(bag_path, topics['image_topic'], topics['odom_topic'], temp_dir)
        sync_df = synchronize_data(img_csv, odom_csv)
        if sync_df.empty:
            print("  -> Skipping bag due to empty synchronized data.")
            if CLEANUP_INTERMEDIATE_FILES: shutil.rmtree(temp_dir)
            continue
            
        # We now pass the configuration variables to the function
        event_timestamps = find_event_windows(sync_df, EVENT_WINDOW_SECONDS, EVENT_THRESHOLD_PERCENTILE)
        print(f"  -> Found {len(event_timestamps)} significant events in this bag.")
        
        for event_ts in event_timestamps:
            create_sliding_window_bundles(sync_df, temp_frames, event_ts, global_bundle_counter)
            print(f"    - Created Sliding Window Bundle b{global_bundle_counter}")
            global_bundle_counter += 1
            
        if CLEANUP_INTERMEDIATE_FILES:
            shutil.rmtree(temp_dir)
            
    print(f"\n{'='*20} PIPELINE COMPLETE {'='*20}")
    print(f"Total Jackal bags processed: {len(jackal_bag_files)}")
    print(f"Total bundles created: {global_bundle_counter - 1}")
    print(f"Sliding window bundles saved in: {OUTPUT_DIR}")

==================== Found 29 Jackal bag files to process ====================

--- Processing Bag: A_Jackal_AHG_Library_Thu_Nov_4_16.bag ---

--- Processing Bag: A_Jackal_AHG_Library_Thu_Oct_28_1.bag ---
  -> Found 9 significant events in this bag.
    - Created Sliding Window Bundle b1
    - Created Sliding Window Bundle b2
    - Created Sliding Window Bundle b3
    - Created Sliding Window Bundle b4
    - Created Sliding Window Bundle b5
    - Created Sliding Window Bundle b6
    - Created Sliding Window Bundle b7
    - Created Sliding Window Bundle b8
    - Created Sliding Window Bundle b9

--- Processing Bag: A_Jackal_AHG_Library_Thu_Oct_28_2.bag ---
  -> Found 8 significant events in this bag.
    - Created Sliding Window Bundle b10
    - Created Sliding Window Bundle b11
    - Created Sliding Window Bundle b12
    - Created Sliding Window Bundle b13
    - Created Sliding Window Bundle b14
    - Created Sliding Window Bundle b15
    - Created Sliding Window Bundle b16
    - Creat

In [ ]:
import shutil
from pathlib import Path

GEMINI_BUNDLES_DIR = Path("./gemini_bundles")
LOCAL_MODEL_BUNDLES_DIR = Path("./local_model_bundles")

def reorganize_bundles_inplace():
    # Finds, sorts, and renames bundles in-place to make them sequential.
    
    print("--- Starting In-Place Bundle Reorganization ---")
    print("🛑 WARNING: This script will rename folders in your original directories.")
    
    # 1. Get the list of original bundle names
    try:
        original_bundle_names = [p.name for p in GEMINI_BUNDLES_DIR.iterdir() if p.is_dir() and p.name.startswith('b')]
    except FileNotFoundError:
        print(f"❌ ERROR: Input directory not found: {GEMINI_BUNDLES_DIR}")
        return

    if not original_bundle_names:
        print("No bundles found to reorganize.")
        return

    # 2. Sort the bundle names numerically (so b7 comes before b10)
    sorted_original_names = sorted(original_bundle_names, key=lambda name: int(name[1:]))
    print(f"\nFound {len(sorted_original_names)} bundles to reorganize.")

    # 3. First Pass: Rename all target folders to a temporary name to avoid conflicts
    print("\nStep 1: Renaming bundles to temporary names...")
    for original_name in sorted_original_names:
        # Define source paths
        source_gemini = GEMINI_BUNDLES_DIR / original_name
        source_local = LOCAL_MODEL_BUNDLES_DIR / original_name
        
        # Define temporary paths
        temp_gemini = GEMINI_BUNDLES_DIR / f"{original_name}_temp"
        temp_local = LOCAL_MODEL_BUNDLES_DIR / f"{original_name}_temp"

        # Rename to temporary
        if source_gemini.exists(): source_gemini.rename(temp_gemini)
        if source_local.exists(): source_local.rename(temp_local)
    print("-> Temporary renaming complete.")

    # 4. Second Pass: Rename temporary folders to their final, sequential names
    print("\nStep 2: Renaming bundles to final sequential names...")
    new_bundle_index = 1
    for original_name in sorted_original_names:
        print(f"-> Mapping {original_name}  ->  b{new_bundle_index}")

        # Define temporary paths
        temp_gemini = GEMINI_BUNDLES_DIR / f"{original_name}_temp"
        temp_local = LOCAL_MODEL_BUNDLES_DIR / f"{original_name}_temp"
        
        # Define final destination paths
        dest_gemini = GEMINI_BUNDLES_DIR / f"b{new_bundle_index}"
        dest_local = LOCAL_MODEL_BUNDLES_DIR / f"b{new_bundle_index}"

        # Rename from temporary to final
        if temp_gemini.exists(): temp_gemini.rename(dest_gemini)
        if temp_local.exists(): temp_local.rename(dest_local)

        new_bundle_index += 1

    print("\n" + "="*30)
    print("✅ Reorganization Complete.")
    print(f"Total bundles processed: {new_bundle_index - 1}")
    print(f"Folders in '{GEMINI_BUNDLES_DIR}' and '{LOCAL_MODEL_BUNDLES_DIR}' are now sequential.")
    print("="*30)


if __name__ == "__main__":
    reorganize_bundles_inplace()

--- Starting In-Place Bundle Reorganization ---
🛑 WARNING: This script will rename folders in your original directories.

Found 50 bundles to reorganize.

Step 1: Renaming bundles to temporary names...
-> Temporary renaming complete.

Step 2: Renaming bundles to final sequential names...
-> Mapping b1  ->  b1
-> Mapping b2  ->  b2
-> Mapping b4  ->  b3
-> Mapping b5  ->  b4
-> Mapping b6  ->  b5
-> Mapping b7  ->  b6
-> Mapping b11  ->  b7
-> Mapping b12  ->  b8
-> Mapping b14  ->  b9
-> Mapping b19  ->  b10
-> Mapping b20  ->  b11
-> Mapping b21  ->  b12
-> Mapping b22  ->  b13
-> Mapping b23  ->  b14
-> Mapping b24  ->  b15
-> Mapping b28  ->  b16
-> Mapping b29  ->  b17
-> Mapping b30  ->  b18
-> Mapping b33  ->  b19
-> Mapping b34  ->  b20
-> Mapping b36  ->  b21
-> Mapping b37  ->  b22
-> Mapping b38  ->  b23
-> Mapping b40  ->  b24
-> Mapping b42  ->  b25
-> Mapping b43  ->  b26
-> Mapping b44  ->  b27
-> Mapping b49  ->  b28
-> Mapping b51  ->  b29
-> Mapping b52  ->  b30
-> Map

In [ ]:
import cv2
import numpy as np
import pandas as pd
import json
import csv
import shutil
from pathlib import Path
from rosbags.highlevel import AnyReader
from scipy.signal import find_peaks


# 1. input directory containing all clean Jackal bag files
INPUT_BAG_DIR = Path("/home/afaris/Final_Project/newly_downloaded_jackal_bags")

# 2. Set the name for the final output directory
OUTPUT_DIR = Path("./pre_event_frame_bundles")

# 3. Parameters
EVENT_WINDOW_SECONDS = 3.0
EVENT_THRESHOLD_PERCENTILE = 95.0
NUM_PREV_FRAMES_INPUT = 5 # As per your new requirement
CLEANUP_INTERMEDIATE_FILES = True


def find_topics(bag_path: Path):
    # Finds the required Jackal topics in a bag file
    with AnyReader([bag_path]) as reader:
        topics_in_bag = {c.topic for c in reader.connections}
    if "/jackal_velocity_controller/odom" in topics_in_bag and "/camera/rgb/image_raw/compressed" in topics_in_bag:
        return {'image_topic': "/camera/rgb/image_raw/compressed", 'odom_topic': "/jackal_velocity_controller/odom"}
    return None

def extract_and_save_data(bag_path, image_topic, odom_topic, temp_dir):
    # Streams odom data to a CSV and saves every compressed image frame to a temporary directory, named by its timestamp.
    
    odom_csv_path = temp_dir / "odom.csv"
    temp_frames_path = temp_dir / "frames"
    temp_frames_path.mkdir(exist_ok=True)
    
    image_timestamps = []

    with open(odom_csv_path, 'w', newline='') as odom_f:
        odom_writer = csv.writer(odom_f)
        odom_writer.writerow(['timestamp', 'linear_x', 'angular_z'])
        with AnyReader([bag_path]) as reader:
            connections = [c for c in reader.connections if c.topic in [image_topic, odom_topic]]
            for conn, ts, rawdata in reader.messages(connections=connections):
                ts_sec = ts / 1e9
                if conn.topic == image_topic:
                    image_timestamps.append(ts_sec)
                    with open(temp_frames_path / f"{ts_sec:.6f}.jpg", 'wb') as img_file:
                        msg = reader.deserialize(rawdata, conn.msgtype)
                        img_file.write(msg.data)
                elif conn.topic == odom_topic:
                    msg = reader.deserialize(rawdata, conn.msgtype)
                    odom_writer.writerow([ts_sec, msg.twist.twist.linear.x, msg.twist.twist.angular.z])

    image_csv_path = temp_dir / "images.csv"
    pd.DataFrame(image_timestamps, columns=['timestamp']).to_csv(image_csv_path, index=False)
    
    return image_csv_path, odom_csv_path, temp_frames_path

def synchronize_data(image_csv_path, odom_csv_path):
    # Synchronizes image and odom data using linear interpolation
    image_df, odom_df = pd.read_csv(image_csv_path), pd.read_csv(odom_csv_path)
    if odom_df.empty or image_df.empty: return pd.DataFrame()
    img_ts, odom_ts, odom_vels = image_df['timestamp'].to_numpy(), odom_df['timestamp'].to_numpy(), odom_df[['linear_x', 'angular_z']].to_numpy()
    sync_data = []
    for t_img in img_ts:
        idx_after = np.searchsorted(odom_ts, t_img)
        idx_before = idx_after - 1
        if idx_before < 0 or idx_after >= len(odom_ts): continue
        t_before, v_before = odom_ts[idx_before], odom_vels[idx_before]
        t_after, v_after = odom_ts[idx_after], odom_vels[idx_after]
        ratio = (t_img - t_before) / (t_after - t_before) if (t_after - t_before) != 0 else 0
        v_interp = v_before + ratio * (v_after - v_before)
        sync_data.append({'timestamp': t_img, 'linear_x': v_interp[0], 'angular_z': v_interp[1]})
    return pd.DataFrame(sync_data)

def find_event_windows(sync_df, event_window_seconds, event_threshold_percentile):
    # Finds all significant, non-overlapping turning events
    if sync_df.empty: return []
    avg_fps = 1.0 / np.mean(np.diff(sync_df['timestamp'])) if len(sync_df) > 1 else 30.0
    
    df = sync_df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.set_index('timestamp', inplace=True)
    rolling_std = df['angular_z'].rolling(f'{event_window_seconds}s').std().dropna()
    if rolling_std.empty: return []
    threshold = np.percentile(rolling_std, event_threshold_percentile)
    distance = int(event_window_seconds * avg_fps)
    peaks, _ = find_peaks(rolling_std, height=threshold, distance=distance)
    return [rolling_std.index[p] for p in peaks]

def extract_pre_event_frames(temp_frames_path, event_end_ts, bundle_id):
    # The first keyframe is the start of the 3-second event window
    event_start_ts = event_end_ts - pd.Timedelta(seconds=EVENT_WINDOW_SECONDS)
    
    bundle_path = OUTPUT_DIR / f"b{bundle_id}"
    bundle_path.mkdir(exist_ok=True)
    
    # Get a sorted list of all saved frame paths and their timestamps
    saved_frame_paths = sorted(list(temp_frames_path.glob("*.jpg")))
    if not saved_frame_paths: return
    saved_frame_timestamps = np.array([float(p.stem) for p in saved_frame_paths])

    # Find the index of the event's first keyframe in the full list of frames
    start_keyframe_idx = np.searchsorted(saved_frame_timestamps, event_start_ts.timestamp())
    
    # Determine the start index for the 5 PREVIOUS frames
    start_idx = max(0, start_keyframe_idx - NUM_PREV_FRAMES_INPUT)
    
    # Get the paths for the frames we need to copy (up to the start keyframe)
    frames_to_copy = saved_frame_paths[start_idx:start_keyframe_idx]
    
    for frame_count, frame_path in enumerate(frames_to_copy):
        dest_name = f"prev_frame_{frame_count}.jpg"
        shutil.copy(frame_path, bundle_path / dest_name)

# MAIN WORKFLOW
if __name__ == "__main__":
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    jackal_bag_files = sorted(list(INPUT_BAG_DIR.glob("*Jackal*.bag")))
    print(f"{'='*20} Found {len(jackal_bag_files)} Jackal bag files to process {'='*20}")
    
    global_bundle_counter = 1
    
    for bag_path in jackal_bag_files:
        print(f"\n--- Processing Bag: {bag_path.name} ---")
        temp_dir = INPUT_BAG_DIR / f"temp_{bag_path.stem}"
        temp_dir.mkdir(exist_ok=True)
        
        topics = find_topics(bag_path)
        if not topics: continue
            
        img_csv, odom_csv, temp_frames = extract_and_save_data(bag_path, topics['image_topic'], topics['odom_topic'], temp_dir)
        sync_df = synchronize_data(img_csv, odom_csv)
        if sync_df.empty:
            if CLEANUP_INTERMEDIATE_FILES: shutil.rmtree(temp_dir)
            continue
            
        event_timestamps = find_event_windows(sync_df, EVENT_WINDOW_SECONDS, EVENT_THRESHOLD_PERCENTILE)
        print(f"  -> Found {len(event_timestamps)} significant events in this bag.")
        
        for event_ts in event_timestamps:
            # Call the new, simplified bundle creation function
            extract_pre_event_frames(temp_frames, event_ts, global_bundle_counter)
            print(f"    - Created Pre-Event Frame Bundle b{global_bundle_counter}")
            global_bundle_counter += 1
            
        if CLEANUP_INTERMEDIATE_FILES:
            shutil.rmtree(temp_dir)
            
    print(f"\n{'='*20} PIPELINE COMPLETE {'='*20}")
    print(f"Total Jackal bags processed: {len(jackal_bag_files)}")
    print(f"Total bundles created: {global_bundle_counter - 1}")
    print(f"Pre-event frame bundles saved in: {OUTPUT_DIR}")

==================== Found 29 Jackal bag files to process ====================

--- Processing Bag: A_Jackal_AHG_Library_Thu_Nov_4_16.bag ---

--- Processing Bag: A_Jackal_AHG_Library_Thu_Oct_28_1.bag ---
  -> Found 9 significant events in this bag.
    - Created Pre-Event Frame Bundle b1
    - Created Pre-Event Frame Bundle b2
    - Created Pre-Event Frame Bundle b3
    - Created Pre-Event Frame Bundle b4
    - Created Pre-Event Frame Bundle b5
    - Created Pre-Event Frame Bundle b6
    - Created Pre-Event Frame Bundle b7
    - Created Pre-Event Frame Bundle b8
    - Created Pre-Event Frame Bundle b9

--- Processing Bag: A_Jackal_AHG_Library_Thu_Oct_28_2.bag ---
  -> Found 8 significant events in this bag.
    - Created Pre-Event Frame Bundle b10
    - Created Pre-Event Frame Bundle b11
    - Created Pre-Event Frame Bundle b12
    - Created Pre-Event Frame Bundle b13
    - Created Pre-Event Frame Bundle b14
    - Created Pre-Event Frame Bundle b15
    - Created Pre-Event Frame Bundle

In [ ]:
import shutil
from pathlib import Path



# 1. path to the parent folder containing our final, curated bundles.
BUNDLES_DIR = Path("/home/afaris/Final_Project/pre_event_frame_bundles") 


def reorganize_bundles_inplace():
    print("--- Starting In-Place Bundle Reorganization ---")
    print(f"🛑 WARNING: This will rename subfolders inside '{BUNDLES_DIR}'.")
    
    if not BUNDLES_DIR.is_dir():
        print(f"❌ ERROR: Input directory not found: {BUNDLES_DIR}")
        return

    # 1. Get the list of original bundle names
    try:
        original_bundle_names = [p.name for p in BUNDLES_DIR.iterdir() if p.is_dir() and p.name.startswith('b')]
    except FileNotFoundError:
        print(f"❌ ERROR: Input directory not found: {BUNDLES_DIR}")
        return

    if not original_bundle_names:
        print("No bundles found to reorganize.")
        return

    # 2. Sort the bundle names numerically (so b7 comes before b10)
    sorted_original_names = sorted(original_bundle_names, key=lambda name: int(name[1:]))
    print(f"\nFound {len(sorted_original_names)} bundles to reorganize.")

    # 3. First Pass: Rename all target folders to a temporary name to avoid conflicts
    print("\nStep 1: Renaming bundles to temporary names...")
    temp_paths = []
    for original_name in sorted_original_names:
        source_path = BUNDLES_DIR / original_name
        temp_path = BUNDLES_DIR / f"{original_name}_temp"
        if source_path.exists():
            source_path.rename(temp_path)
            temp_paths.append(temp_path)
    print("-> Temporary renaming complete.")

    # 4. Second Pass: Rename temporary folders to their final, sequential names
    print("\nStep 2: Renaming bundles to final sequential names...")
    new_bundle_index = 1
    for temp_path in temp_paths:
        dest_path = BUNDLES_DIR / f"b{new_bundle_index}"
        
        # Get the original name back for the print statement
        original_name = temp_path.name.replace('_temp', '')
        print(f"-> Mapping {original_name}  ->  b{new_bundle_index}")

        if temp_path.exists():
            temp_path.rename(dest_path)

        new_bundle_index += 1

    print("\n" + "="*30)
    print("✅ Reorganization Complete.")
    print(f"Total bundles processed: {new_bundle_index - 1}")
    print(f"Folders in '{BUNDLES_DIR}' are now sequential.")
    print("="*30)


if __name__ == "__main__":
    reorganize_bundles_inplace()

--- Starting In-Place Bundle Reorganization ---
🛑 WARNING: This will rename subfolders inside '/home/afaris/Final_Project/pre_event_frame_bundles'.

Found 50 bundles to reorganize.

Step 1: Renaming bundles to temporary names...
-> Temporary renaming complete.

Step 2: Renaming bundles to final sequential names...
-> Mapping b1  ->  b1
-> Mapping b2  ->  b2
-> Mapping b4  ->  b3
-> Mapping b5  ->  b4
-> Mapping b6  ->  b5
-> Mapping b7  ->  b6
-> Mapping b11  ->  b7
-> Mapping b12  ->  b8
-> Mapping b14  ->  b9
-> Mapping b19  ->  b10
-> Mapping b20  ->  b11
-> Mapping b21  ->  b12
-> Mapping b22  ->  b13
-> Mapping b23  ->  b14
-> Mapping b24  ->  b15
-> Mapping b28  ->  b16
-> Mapping b29  ->  b17
-> Mapping b30  ->  b18
-> Mapping b33  ->  b19
-> Mapping b34  ->  b20
-> Mapping b36  ->  b21
-> Mapping b37  ->  b22
-> Mapping b38  ->  b23
-> Mapping b40  ->  b24
-> Mapping b42  ->  b25
-> Mapping b43  ->  b26
-> Mapping b44  ->  b27
-> Mapping b49  ->  b28
-> Mapping b51  ->  b29
-> 

In [ ]:
import json
import random
from pathlib import Path


# 1. Path to the bundles with the 6 keyframes and original commands.json
KEYFRAME_DATA_DIR = Path("/home/afaris/Final_Project/gemini_bundles") 

# 2. Path to the bundles with the 5 frames that come BEFORE the first keyframe
PRE_EVENT_FRAMES_DIR = Path("/home/afaris/Final_Project/pre_event_frame_bundles")

# 3. Path to the folder with the final, rephrased text files (e.g., b1_final.json)
FINAL_TEXT_DIR = Path("/home/afaris/Final_Project/final_training_data_source1")

# 4. Set the names for the output dataset files
TRAINING_OUTPUT_FILE = Path("./training_dataset6.jsonl")
VALIDATION_OUTPUT_FILE = Path("./validation_dataset6.jsonl")

# 5. Set the ratio for the validation split (e.g., 0.2 = 20%)
VALIDATION_SPLIT_RATIO = 0.2

# 6. Thresholds for converting velocities to high-level actions
ACTION_THRESHOLDS = {
    'stop': 0.1,
    'turn': 0.25
}


def get_high_level_action(linear_x, angular_z, thresholds):
    # Converts numerical velocities into a categorical action label.
    if abs(linear_x) < thresholds['stop'] and abs(angular_z) < thresholds['turn']:
        return "stop"
    if angular_z < -thresholds['turn']:
        return "go_right"
    if angular_z > thresholds['turn']:
        return "go_left"
    if linear_x < -thresholds['stop']:
        return "go_backward"
    return "go_forward"

def create_training_dataset():

    # Generates the final training and validation dataset files based on the final sliding window plane
    print("--- Starting Final Dataset Creation ---")

    bundle_paths = sorted(
        [p for p in KEYFRAME_DATA_DIR.iterdir() if p.is_dir() and p.name.startswith('b')],
        key=lambda p: int(p.name[1:])
    )

    if not bundle_paths:
        print(f"❌ ERROR: No bundle folders found in '{KEYFRAME_DATA_DIR}'.")
        return

    all_examples = []
    print(f"Found {len(bundle_paths)} bundles to process...")

    for bundle_path in bundle_paths:
        bundle_name = bundle_path.name
        
        try:
            # 1. Load all necessary data sources for this bundle 
            # Load the final rephrased text from the new directory
            with open(FINAL_TEXT_DIR / f"{bundle_name}_final.json", 'r') as f:
                rephrased_analysis = json.load(f)
            with open(bundle_path / 'commands.json', 'r') as f:
                original_commands = json.load(f)
            
            # Get paths to the 6 keyframes from the main bundle folder
            keyframe_paths = [bundle_path / f"frame_{i}.jpg" for i in range(6)]

            # Get paths to the 5 frames before the event started
            pre_event_frames_path = PRE_EVENT_FRAMES_DIR / bundle_name
            pre_event_frame_paths = sorted(list(pre_event_frames_path.glob("*.jpg")))

            # Validate that all files exist before processing
            if len(pre_event_frame_paths) < 5:
                raise FileNotFoundError(f"Fewer than 5 pre-event frames found in {pre_event_frames_path}")
            for p in keyframe_paths:
                if not p.is_file(): raise FileNotFoundError(f"Missing keyframe: {p}")

        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"⚠️ WARNING: Skipping bundle {bundle_name} due to missing or corrupt files: {e}")
            continue

        # 2. Create 5 training examples (predicting for keyframes 2 through 6) 
        for i in range(1, 6): # This will loop for i = 1, 2, 3, 4, 5
            
            keyframe_to_predict_idx = i

            # Assemble the INPUT frames for the sliding window 
            num_keyframes_from_event = i
            num_pre_event_frames = 6 - num_keyframes_from_event
            input_frame_paths = (
                pre_event_frame_paths[5 - num_pre_event_frames:] + 
                keyframe_paths[:num_keyframes_from_event]
            )

            if len(input_frame_paths) != 6:
                print(f"⚠️ WARNING: Could not construct 6 input frames for {bundle_name} keyframe {i+1}. Skipping.")
                continue

            # Assemble the GROUND TRUTH "answer" for the keyframe we are predicting 
            gt_command = original_commands.get(f'frame_{keyframe_to_predict_idx}.jpg', {})
            gt_linear_x = gt_command.get('linear_x', 0)
            gt_angular_z = gt_command.get('angular_z', 0)
            gt_action = get_high_level_action(gt_linear_x, gt_angular_z, ACTION_THRESHOLDS)

            # Get GT text descriptions from the rephrased analysis files
            desc_analysis_block = rephrased_analysis[i-1]
            explanation_analysis_block = rephrased_analysis[min(i, len(rephrased_analysis) - 1)]

            ground_truth_output = {
                "Description of the scene": desc_analysis_block.get("scene_description", ""),
                "Description of the future robot motion": gt_action,
                "Explanation of the future robot motion": explanation_analysis_block.get("action_justification", ""),
                "Cmd vel for the next 0.5 second": {
                    "linear_velocity": gt_linear_x,
                    "angular_velocity": gt_angular_z
                }
            }

            # Assemble the final training example 
            prompt_for_model = "You are a robot. Analyze the following image sequence to decide your next action. Provide your response in a single JSON object."
            
            training_example = {
                "images": [str(p) for p in input_frame_paths],
                "text": f"USER: {prompt_for_model}\nASSISTANT: {json.dumps(ground_truth_output)}"
            }
            all_examples.append(training_example)

    print(f"\nSuccessfully created {len(all_examples)} total training examples.")

    # Shuffle and Split the Dataset 
    if not all_examples:
        print("No examples were created. Exiting.")
        return
        
    random.shuffle(all_examples)
    split_index = int(len(all_examples) * (1 - VALIDATION_SPLIT_RATIO))
    train_set = all_examples[:split_index]
    validation_set = all_examples[split_index:]

    print(f"Splitting data into {len(train_set)} training and {len(validation_set)} validation examples.")

    # Write to JSONL files 
    with open(TRAINING_OUTPUT_FILE, 'w') as f:
        for example in train_set:
            f.write(json.dumps(example) + '\n')
            
    with open(VALIDATION_OUTPUT_FILE, 'w') as f:
        for example in validation_set:
            f.write(json.dumps(example) + '\n')

    print("\n" + "="*50)
    print("✅ Final Dataset Creation Complete.")
    print(f"Training data saved to: {TRAINING_OUTPUT_FILE}")
    print(f"Validation data saved to: {VALIDATION_OUTPUT_FILE}")
    print("="*50)

if __name__ == "__main__":
    create_training_dataset()

--- Starting Final Dataset Creation ---
Found 50 bundles to process...

Successfully created 250 total training examples.
Splitting data into 200 training and 50 validation examples.

✅ Final Dataset Creation Complete.
Training data saved to: training_dataset6.jsonl
Validation data saved to: validation_dataset6.jsonl


: 

In [ ]:
import csv
from pathlib import Path
from rosbags.highlevel import AnyReader

base_dir    = Path("/home/afaris/Final_Project")
labels      = [
    "Against Traffic",
    "Navigating Through Large Crowds",
    "overtaking",
    "Passing Conversational Groups",
]
primary_topic   = "/jackal_velocity_controller/odom"
fallback_topic  = "/odom"
bundle_root     = base_dir / "BUNDLE"

bundle_root.mkdir(exist_ok=True)

def extract_velocities(bag_path: Path, out_csv: Path, topic: str) -> bool:
    records = []
    with AnyReader([bag_path]) as reader:
        conns = [c for c in reader.connections if c.topic == topic]
        if not conns:
            return False
        for conn, ts, raw in reader.messages(conns):
            msg = reader.deserialize(raw, conn.msgtype)

            # timestamp
            try:
                t_abs = msg.header.stamp.sec + msg.header.stamp.nanosec * 1e-9
            except AttributeError:
                # twist‐only fallback on reader timestamp
                if hasattr(ts, "sec") and hasattr(ts, "nanosec"):
                    t_abs = ts.sec + ts.nanosec * 1e-9
                elif hasattr(ts, "seconds") and hasattr(ts, "nanoseconds"):
                    t_abs = ts.seconds + ts.nanoseconds * 1e-9
                else:
                    t_abs = ts * 1e-9

            # extract forward & yaw
            lin_x = msg.twist.twist.linear.x if hasattr(msg, "twist") else msg.linear.x
            ang_z = msg.twist.twist.angular.z if hasattr(msg, "twist") else msg.angular.z
            records.append((t_abs, lin_x, ang_z))

    if not records:
        return False

    # relative time from first message
    t0 = records[0][0]
    rel = [(t - t0, lin, ang) for (t, lin, ang) in records]

    # write CSV
    with out_csv.open("w", newline="") as fh:
        writer = csv.writer(fh)
        writer.writerow(["time_s", "linear_x", "angular_z"])
        writer.writerows(rel)

    return True


# MAIN LOOP 
for label in labels:
    in_folder  = base_dir / label / "bag files"
    out_folder = bundle_root / label
    out_folder.mkdir(parents=True, exist_ok=True)

    bag_files = sorted(in_folder.glob("*.bag"))
    print(f"\n⏳ Processing '{label}' ({len(bag_files)} bags)")

    for idx, bag in enumerate(bag_files, start=1):
        # peek available topics
        with AnyReader([bag]) as rdr:
            available = {c.topic for c in rdr.connections}

        # pick primary or fallback
        if primary_topic in available:
            topic = primary_topic
        elif fallback_topic in available:
            topic = fallback_topic
        else:
            print(f"  ⚠ Skipping {bag.name}: neither odom topic present")
            continue

        out_csv = out_folder / f"{bag.stem}_Bundle{idx}.csv"
        ok = extract_velocities(bag, out_csv, topic)
        status = "✅" if ok else "❌"
        print(f"  {status} {bag.name} → {out_csv.name}  (used '{topic}')")



⏳ Processing 'Against Traffic' (5 bags)
  ✅ A_Jackal_AHG_Library_Thu_Oct_28_1.bag → A_Jackal_AHG_Library_Thu_Oct_28_1_Bundle1.csv  (used '/jackal_velocity_controller/odom')
  ✅ A_Jackal_AHG_Library_Thu_Oct_28_2.bag → A_Jackal_AHG_Library_Thu_Oct_28_2_Bundle2.csv  (used '/jackal_velocity_controller/odom')
  ✅ A_Jackal_Library_AHG_Thu_Nov_4_17.bag → A_Jackal_Library_AHG_Thu_Nov_4_17_Bundle3.csv  (used '/jackal_velocity_controller/odom')
  ✅ A_Spot_Boatbuilding_AHG_Thu_Nov_4_23.bag → A_Spot_Boatbuilding_AHG_Thu_Nov_4_23_Bundle4.csv  (used '/odom')
  ✅ A_Spot_Library_AHG_Thu_Nov_4_22.bag → A_Spot_Library_AHG_Thu_Nov_4_22_Bundle5.csv  (used '/odom')

⏳ Processing 'Navigating Through Large Crowds' (10 bags)
  ✅ A_Spot_Butler_Foodtruck_Fri_Nov_26_132.bag → A_Spot_Butler_Foodtruck_Fri_Nov_26_132_Bundle1.csv  (used '/odom')
  ✅ A_Spot_Butler_LBJ_Sat_Nov_13_104.bag → A_Spot_Butler_LBJ_Sat_Nov_13_104_Bundle2.csv  (used '/odom')
  ✅ A_Spot_EER_OsCafe_Tue_Nov_9_39.bag → A_Spot_EER_OsCafe_Tue_Nov_9

In [ ]:
import subprocess
from pathlib import Path

base_in    = Path("/home/afaris/Final_Project/segments")
base_out   = Path("/home/afaris/Final_Project/Segments")
labels     = [
    "against traffic",
    "Navigating through large crowds",
    "overtaking",
    "Passing Conversational Groups",
]
target_fps = 29.97

# Ensure output root exists
base_out.mkdir(parents=True, exist_ok=True)

for label in labels:
    in_dir  = base_in  / label
    out_dir = base_out / label
    out_dir.mkdir(parents=True, exist_ok=True)
    print(f"\nProcessing '{label}' → '{out_dir}'")

    # Only look for .mp4 files
    for vid in sorted(in_dir.glob("*.mp4")):
        out_name = f"{vid.stem}#{vid.suffix}"
        out_path = out_dir / out_name

        print(f"  ↳ {vid.name}  →  {out_name}  @ {target_fps}fps")
        subprocess.run([
            "ffmpeg",
            "-y",                    # overwrite
            "-i", str(vid),          # input file
            "-r", str(target_fps),   # set output fps
            "-c:v", "libx264",       # H.264 encoding
            "-pix_fmt", "yuv420p",   # ensure broad compatibility
            str(out_path)
        ], check=True)

print("\n✅ All .mp4 videos re-encoded.") 



Processing 'against traffic' → '/home/afaris/Final_Project/Segments/against traffic'
  ↳ AT1-00.00.10.154-00.00.12.723-Against_Traffic__bike_going_from_the_side.mp4  →  AT1-00.00.10.154-00.00.12.723-Against_Traffic__bike_going_from_the_side#.mp4  @ 29.97fps


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

  ↳ AT1-00.00.26.873-00.00.30.794-Against_Traffic__Bike_coming_from_the_front.mp4  →  AT1-00.00.26.873-00.00.30.794-Against_Traffic__Bike_coming_from_the_front#.mp4  @ 29.97fps


frame=  121 fps= 88 q=-1.0 Lsize=    5321kB time=00:00:03.93 bitrate=11071.6kbits/s dup=0 drop=119 speed=2.85x    
video:5319kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.042135%
[libx264 @ 0x5fff51e402c0] frame I:1     Avg QP:27.51  size:116937
[libx264 @ 0x5fff51e402c0] frame P:48    Avg QP:28.46  size: 59020
[libx264 @ 0x5fff51e402c0] frame B:72    Avg QP:29.93  size: 34670
[libx264 @ 0x5fff51e402c0] consecutive B-frames: 15.7%  9.9% 14.9% 59.5%
[libx264 @ 0x5fff51e402c0] mb I  I16..4:  1.0% 88.6% 10.4%
[libx264 @ 0x5fff51e402c0] mb P  I16..4:  1.6% 25.5%  5.6%  P16..4: 26.4% 27.7% 12.2%  0.0%  0.0%    skip: 1.0%
[libx264 @ 0x5fff51e402c0] mb B  I16..4:  1.2% 10.3%  2.0%  B16..8: 38.8% 28.3%  6.2%  direct: 5.6%  skip: 7.7%  L0:48.7% L1:24.0% BI:27.3%
[libx264 @ 0x5fff51e402c0] 8x8 transform intra:77.9% inter:75.7%
[libx264 @ 0x5fff51e402c0] coded y,uvDC,uvAC intra: 79.4% 57.1% 7.4% inter: 57.2% 34.7% 0.3%
[libx264 @ 0x5fff51e402c0] i16 v,h,dc,p: 1

  ↳ AT1-00.01.00.797-00.01.04.766-Against_Traffic__Bike_coming_from_the_front.mp4  →  AT1-00.01.00.797-00.01.04.766-Against_Traffic__Bike_coming_from_the_front#.mp4  @ 29.97fps


...

frame=  181 fps=117 q=-1.0 Lsize=    3753kB time=00:00:05.93 bitrate=5176.1kbits/s dup=0 drop=178 speed=3.84x    
video:3750kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.073410%
[libx264 @ 0x59fc4647e580] frame I:1     Avg QP:25.59  size: 35125
[libx264 @ 0x59fc4647e580] frame P:77    Avg QP:26.30  size: 27320
[libx264 @ 0x59fc4647e580] frame B:103   Avg QP:27.82  size: 16510
[libx264 @ 0x59fc4647e580] consecutive B-frames: 20.4%  5.5% 16.6% 57.5%
[libx264 @ 0x59fc4647e580] mb I  I16..4:  7.5% 86.1%  6.4%
[libx264 @ 0x59fc4647e580] mb P  I16..4:  6.2% 35.8%  3.9%  P16..4: 30.2% 12.1%  4.2%  0.0%  0.0%    skip: 7.7%
[libx264 @ 0x59fc4647e580] mb B  I16..4:  2.9% 12.1%  1.0%  B16..8: 39.1% 13.2%  2.5%  direct: 5.4%  skip:23.7%  L0:48.3% L1:39.4% BI:12.3%
[libx264 @ 0x59fc4647e580] 8x8 transform intra:77.4% inter:77.8%
[libx264 @ 0x59fc4647e580] coded y,uvDC,uvAC intra: 55.4% 61.1% 15.3% inter: 30.1% 30.4% 0.7%
[libx264 @ 0x59fc4647e580] i16 v,h,dc,p: 3

  ↳ 8-00.00.08.623-00.00.10.608-seg1.mp4  →  8-00.00.08.623-00.00.10.608-seg1#.mp4  @ 29.97fps


frame=   62 fps=0.0 q=-1.0 Lsize=    1248kB time=00:00:01.96 bitrate=5192.1kbits/s dup=0 drop=58 speed=3.18x    
video:1246kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.127499%
[libx264 @ 0x6223f4342e00] frame I:1     Avg QP:25.64  size: 33721
[libx264 @ 0x6223f4342e00] frame P:16    Avg QP:26.23  size: 27755
[libx264 @ 0x6223f4342e00] frame B:45    Avg QP:27.09  size: 17724
[libx264 @ 0x6223f4342e00] consecutive B-frames:  1.6%  3.2%  4.8% 90.3%
[libx264 @ 0x6223f4342e00] mb I  I16..4:  4.5% 91.9%  3.6%
[libx264 @ 0x6223f4342e00] mb P  I16..4:  8.2% 50.7%  4.4%  P16..4: 24.2%  8.4%  2.1%  0.0%  0.0%    skip: 1.9%
[libx264 @ 0x6223f4342e00] mb B  I16..4:  2.4% 16.2%  1.0%  B16..8: 39.6% 11.4%  2.0%  direct: 7.1%  skip:20.3%  L0:53.2% L1:39.4% BI: 7.4%
[libx264 @ 0x6223f4342e00] 8x8 transform intra:81.7% inter:83.1%
[libx264 @ 0x6223f4342e00] coded y,uvDC,uvAC intra: 63.6% 54.9% 8.2% inter: 33.0% 35.8% 0.3%
[libx264 @ 0x6223f4342e00] i16 v,h,dc,p: 23%

  ↳ 8-00.00.12.136-00.00.14.351-seg2.mp4  →  8-00.00.12.136-00.00.14.351-seg2#.mp4  @ 29.97fps

✅ All .mp4 videos re-encoded.


frame=   72 fps=0.0 q=-1.0 Lsize=    1572kB time=00:00:02.30 bitrate=5593.4kbits/s dup=0 drop=69 speed=3.25x    
video:1570kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.108145%
[libx264 @ 0x595cdebb4440] frame I:1     Avg QP:25.44  size: 41669
[libx264 @ 0x595cdebb4440] frame P:20    Avg QP:26.38  size: 29589
[libx264 @ 0x595cdebb4440] frame B:51    Avg QP:27.37  size: 19096
[libx264 @ 0x595cdebb4440] consecutive B-frames:  5.6%  0.0%  0.0% 94.4%
[libx264 @ 0x595cdebb4440] mb I  I16..4:  4.5% 92.3%  3.2%
[libx264 @ 0x595cdebb4440] mb P  I16..4:  6.2% 45.5%  4.0%  P16..4: 29.4%  9.8%  2.8%  0.0%  0.0%    skip: 2.3%
[libx264 @ 0x595cdebb4440] mb B  I16..4:  2.4% 15.3%  1.0%  B16..8: 41.1% 11.9%  2.1%  direct: 7.5%  skip:18.7%  L0:51.8% L1:39.4% BI: 8.8%
[libx264 @ 0x595cdebb4440] 8x8 transform intra:82.2% inter:82.4%
[libx264 @ 0x595cdebb4440] coded y,uvDC,uvAC intra: 66.1% 50.4% 4.7% inter: 35.9% 34.5% 0.2%
[libx264 @ 0x595cdebb4440] i16 v,h,dc,p: 23%

In [ ]:
import re
import csv
import shutil
from pathlib import Path
from typing import Tuple

base_dir    = Path("/home/afaris/Final_Project")
bundle_root = base_dir / "BUNDLE"
output_root = base_dir / "MERGED"
labels      = [
    "Against Traffic",
    "Navigating Through Large Crowds",
    "overtaking",
    "Passing Conversational Groups",
]

def parse_times(name: str) -> Tuple[float,float]:
    stamps = re.findall(r"\d{2}\.\d{2}\.\d{2}\.\d{3}", name)
    if len(stamps) < 2:
        raise ValueError(f"No two timestamps in segment name '{name}'")
    def to_sec(st: str) -> float:
        h,m,s,ms = map(int, st.split('.'))
        return h*3600 + m*60 + s + ms/1000
    return to_sec(stamps[0]), to_sec(stamps[1])

# Ensure output root exists
output_root.mkdir(parents=True, exist_ok=True)

for label in labels:
    in_label_dir  = bundle_root / label
    out_label_dir = output_root / label
    print(f"\n🔍 Label '{label}'")
    if not in_label_dir.exists():
        print("  ⚠ No such folder:", in_label_dir)
        continue

    # process each folder
    for bfolder in sorted(in_label_dir.iterdir()):
        if not bfolder.is_dir() or not bfolder.name.startswith("b"):
            continue

        csv_files = list(bfolder.glob("*.csv"))
        vids      = list(bfolder.glob("*.mp4"))
        if not csv_files or not vids:
            print(f"  ⚠ Skipping {bfolder.name}: missing CSV or MP4")
            continue

        full_csv = csv_files[0]
        # read the full CSV into memory
        rows = []
        with full_csv.open() as f:
            reader = csv.reader(f)
            header = next(reader)
            time_idx = header.index("time_s")
            rows = [row for row in reader]

        print(f"  ▶ {bfolder.name}: {len(vids)} segments")

        for vid in sorted(vids):
            stem = vid.stem
            try:
                t0, t1 = parse_times(stem)
            except ValueError as e:
                print("    ⚠", e)
                continue

            # slice rows between t0 ≤ time_s ≤ t1
            sliced = [
                row for row in rows
                if t0 <= float(row[time_idx]) <= t1
            ]

            # prepare output folder
            out_folder = out_label_dir / bfolder.name / stem
            out_folder.mkdir(parents=True, exist_ok=True)

            # copy the video
            shutil.copy2(vid, out_folder / vid.name)

            # write sliced CSV
            out_csv = out_folder / f"{stem}.csv"
            with out_csv.open("w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(header)
                writer.writerows(sliced)

            print(f"    ✓ {stem}: {len(sliced)} rows → {out_csv.relative_to(base_dir)}")

print("\n✅ Done bundling segments with their velocity snippets.")



🔍 Label 'Against Traffic'
  ▶ b1: 8 segments
    ✓ AT1-00.00.10.154-00.00.12.723-Against_Traffic__bike_going_from_the_side#: 129 rows → MERGED/Against Traffic/b1/AT1-00.00.10.154-00.00.12.723-Against_Traffic__bike_going_from_the_side#/AT1-00.00.10.154-00.00.12.723-Against_Traffic__bike_going_from_the_side#.csv
    ✓ AT1-00.00.26.873-00.00.30.794-Against_Traffic__Bike_coming_from_the_front#: 196 rows → MERGED/Against Traffic/b1/AT1-00.00.26.873-00.00.30.794-Against_Traffic__Bike_coming_from_the_front#/AT1-00.00.26.873-00.00.30.794-Against_Traffic__Bike_coming_from_the_front#.csv
    ✓ AT1-00.01.00.797-00.01.04.766-Against_Traffic__Bike_coming_from_the_front#: 199 rows → MERGED/Against Traffic/b1/AT1-00.01.00.797-00.01.04.766-Against_Traffic__Bike_coming_from_the_front#/AT1-00.01.00.797-00.01.04.766-Against_Traffic__Bike_coming_from_the_front#.csv
    ✓ AT1-00.01.17.205-00.01.18.964-Against_Traffic__Bike_coming_from_the_front#: 88 rows → MERGED/Against Traffic/b1/AT1-00.01.17.205-00.01.

In [ ]:
from pathlib import Path
from rosbags.highlevel import AnyReader

bag_path = Path("/home/afaris/Final_Project/overtaking/bag files/A_Spot_Bass_Rec_Fri_Nov_26_126.bag")
vel_topic = "/odom" 

timestamps = []

# Read velocity command timestamps
with AnyReader([bag_path]) as reader:
    conns = [c for c in reader.connections if c.topic == vel_topic]
    if not conns:
        raise RuntimeError(f"❌ Topic {vel_topic!r} not found in bag file.")

    for conn, ts, _ in reader.messages(conns):
        timestamps.append(ts)

# Convert timestamps to seconds and calculate FPS
if len(timestamps) < 2:
    print("❌ Not enough messages to calculate velocity rate.")
else:
    duration = (timestamps[-1] - timestamps[0]) * 1e-9  # nanoseconds → seconds
    rate = len(timestamps) / duration
    print(f"✅ Velocity Messages: {len(timestamps)}")
    print(f"⏱️ Duration (s): {duration:.2f}")
    print(f"📈 Average Velocity Command Rate (Hz): {rate:.2f}")


✅ Velocity Messages: 8276
⏱️ Duration (s): 505.48
📈 Average Velocity Command Rate (Hz): 16.37
